# Project 2: Market-basket analysis - IMDB dataset

Project for the course of Algorithms for Massive Dataset <br> Nicolas Facchinetti 961648 <br> Antonio Belotti 960822

# Download the dataset from Kaggle

First install the Python module of Kaggle to download the dataset from its datacenter

In [9]:
!pip install kaggle

Then load kaggle.json, a file containing your API credentials to be able to use the services offered by Kaggle

In [10]:
from google.colab import files

uploaded = files.upload()
  
# Move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


Now we can download the dataset

In [3]:
!kaggle datasets download 'ashirwadsangwan/imdb-dataset'

  1%|▌                                     | 20.0M/1.44G [00:18<22:09, 1.15MB/s]^C
  1%|▌                                     | 20.0M/1.44G [00:18<22:54, 1.11MB/s]
User cancelled operation


We now must unzip the compressed archive to use it. Once done we can also remove it.

In [12]:
!unzip imdb-dataset.zip && rm imdb-dataset.zip

Archive:  imdb-dataset.zip
  inflating: name.basics.tsv.gz      
  inflating: name.basics.tsv/name.basics.tsv  
  inflating: title.akas.tsv.gz       
  inflating: title.akas.tsv/title.akas.tsv  
  inflating: title.basics.tsv.gz     
  inflating: title.basics.tsv/title.basics.tsv  
  inflating: title.principals.tsv.gz  
  inflating: title.principals.tsv/title.principals.tsv  
  inflating: title.ratings.tsv.gz    
  inflating: title.ratings.tsv/title.ratings.tsv  


Lets try to load some data in a Pandas Dataframe

In [1]:
import pandas as pd

In [3]:
actors_cols = {
    "original": [
        "nconst",  # actor unique id
        "knownForTitles"  # move he/she is in
    ],
    "renamed": ["actorId", "titles"]
}

actors_df = pd.read_csv(
    "imdb-dataset/name.basics.tsv.gz",
    compression="gzip",
    sep='\t',
    usecols=actors_cols["original"]
)

# clean and pre-process
actors_df.columns = actors_cols["renamed"]
actors_df.drop(actors_df[actors_df.titles == "\\N"].index, inplace=True)
actors_df.titles = actors_df.titles.apply(lambda x: x.split(","))

In [4]:
def apriori(transactions, support_threshold):
    singleton_counter = []
    lookup_index_table = {}
    reverse_lookup_index_table = {}

    # count singletons
    for bucket in transactions:
        for elem in bucket:
            if elem not in lookup_index_table:
                # The newly discovered element is appended on the tail of the array counter
                lookup_index_table[elem] = len(singleton_counter)
                reverse_lookup_index_table[len(singleton_counter)] = elem
                singleton_counter.append(0)

            idx = lookup_index_table[elem]
            singleton_counter[idx] += 1

    frequent_items_table = [i for i,v in enumerate(singleton_counter) if v > support_threshold]

    # count pairs
    pair_counter = {}
    for bucket in transactions:
        frequent_items = [lookup_index_table[item] for item in bucket 
                          if lookup_index_table[item] in frequent_items_table]

        for x in frequent_items:
            for y in frequent_items:
                if x<y:
                    pair_counter[(x,y)] = pair_counter.get((x,y), 0) +1 

    return [list(map(lambda x: reverse_lookup_index_table[x], i)) for i,c in pair_counter.items() 
            if c > support_threshold] 

In [ ]:
# test
rules = apriori(actors_df.titles, 300)

movies_df = pd.read_csv("imdb-dataset/title.basics.tsv.gz", compression='gzip', sep='\t')
from IPython.display import display

for x,y in rules:
    display(movies_df.loc[((movies_df.tconst == x) | (movies_df.tconst == y))])

/home/anto/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
117965,tt0120737,movie,The Lord of the Rings: The Fellowship of the Ring,The Lord of the Rings: The Fellowship of the Ring,0,2001,\N,178,"Adventure,Drama,Fantasy"
161829,tt0167260,movie,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Return of the King,0,2003,\N,201,"Adventure,Drama,Fantasy"


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
161829,tt0167260,movie,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Return of the King,0,2003,\N,201,"Adventure,Drama,Fantasy"
161830,tt0167261,movie,The Lord of the Rings: The Two Towers,The Lord of the Rings: The Two Towers,0,2002,\N,179,"Adventure,Drama,Fantasy"


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
117965,tt0120737,movie,The Lord of the Rings: The Fellowship of the Ring,The Lord of the Rings: The Fellowship of the Ring,0,2001,\N,178,"Adventure,Drama,Fantasy"
161830,tt0167261,movie,The Lord of the Rings: The Two Towers,The Lord of the Rings: The Two Towers,0,2002,\N,179,"Adventure,Drama,Fantasy"


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
3674370,tt4154756,movie,Avengers: Infinity War,Avengers: Infinity War,0,2018,\N,149,"Action,Adventure,Sci-Fi"
3674389,tt4154796,movie,Avengers: Endgame,Avengers: Endgame,0,2019,\N,181,"Action,Adventure,Drama"


...

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
273009,tt0284702,tvSeries,Big Brother Uncut,Big Brother Uncut,0,2001,2013,60,Reality-TV
275403,tt0287196,tvSeries,Big Brother: Australia,Big Brother,0,2001,2014,25,"Game-Show,Music,Reality-TV"


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
86667,tt0088580,tvSeries,Neighbours,Neighbours,0,1985,\N,22,"Drama,Romance"
106362,tt0108709,tvSeries,Blue Heelers,Blue Heelers,0,1994,2006,60,"Action,Crime,Drama"


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
64045,tt0065323,tvSeries,NFL Monday Night Football,NFL Monday Night Football,0,1970,\N,\N,Sport
391224,tt0407424,tvSeries,The NFL on NBC,The NFL on NBC,0,1965,1998,\N,Sport


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
3160174,tt2974350,videoGame,Super Smash Bros. for Wii U,Dairantou sumasshu burazâzu for Wii U,0,2014,\N,\N,"Action,Adventure,Family"
3345498,tt3408266,videoGame,Super Smash Bros. for Nintendo 3DS,Dairantou sumasshu burazâzu for Nintendo 3DS,0,2014,\N,\N,"Action,Family,Fantasy"


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
64045,tt0065323,tvSeries,NFL Monday Night Football,NFL Monday Night Football,0,1970,\N,\N,Sport
391223,tt0407423,tvSeries,The NFL on CBS,The NFL on CBS,0,1956,\N,\N,Sport
